In [25]:
import numpy as np
import pylab as plt
from math import log10, cos
from pymc import Uniform, MCMC, deterministic

from scipy.interpolate import interp1d as interp
from scipy.optimize import brentq, minimize_scalar, basinhopping
from profiles import build_profile, build_kernel
from dispersion import SphericalJeansDispersion
from likelihood import GaussianLikelihood
#from fitter import *

%matplotlib inline

In [2]:
directory = '/home/andrea/Desktop/work/DWARF/dsphsim/'
rh = 0.04
D = 39.81
theta = 2*rh/D

In [3]:
dm = build_profile('NFW')
st = build_profile('plummer',**{'rh':rh}) # Plummer Stellar profile
kr = build_kernel('iso') # isotropic kernel
dwarf_props = {'D':D, 'theta':theta, 'rt':np.inf, 'with_errs':False}
Sigma = SphericalJeansDispersion(dm, st, kr, dwarf_props)

In [4]:
#R, v = np.loadtxt(directory+'Ret2_data/dsph_001.txt',usecols=(5, 7),unpack=True)
R, v = np.load('results/equiRdSphs.npy')
vnan = np.isnan(v)
v = v[~vnan]
R = R[~vnan]
dv = np.zeros_like(v)

In [5]:
LL = GaussianLikelihood([R, v, dv, 0.], Sigma)

In [6]:
LL.set_free('dm_a')
LL.set_free('dm_b')
LL.set_free('dm_c')
LL.set_free('dm_r0')

$r_s = 0.35 , \rho_s = 7.55 \times 10^7 , J = 16.72$

In [7]:
LL.free_pars.keys()

['dm_a', 'dm_b', 'J', 'dm_r0', 'dm_c']

In [61]:
def loglike(x, J):
    L = LL(x[0], x[1], J, x[3], x[2])
    if not np.isnan(L):
        return L
    else:
        return 1e8

In [62]:
J = 16.

In [63]:
x0 = np.array( [1. ,3. ,1. ,0.2] )

In [ ]:
basinhopping(loglike, x0, niter=1000, T=1.0, stepsize=0.1, minimizer_kwargs={'method':'BFGS','args':J}, take_step=None, accept_test=None, callback=None, interval=50, disp=True, niter_success=None)